### **Bootstrap & imports**

In [1]:
from pathlib import Path
import sys, os, platform, uuid, json, sqlite3, time
print("Python:", platform.python_version())

CWD  = Path.cwd().resolve()
ROOT = CWD if (CWD / "src").exists() else CWD.parent
if str(ROOT) not in sys.path: sys.path.append(str(ROOT))
print("ROOT:", ROOT)

# hot-reload helpers 
import importlib
import src.agent.config as _cfg
import src.agent.orchestrator as _orch
import src.agent.prompts as _prompts
import src.agent.logger as _logger
importlib.reload(_cfg); importlib.reload(_orch); importlib.reload(_prompts); importlib.reload(_logger)

from src.agent.config import CFG
from src.agent.orchestrator import Title17Agent
from src.agent.logger import EventLogger


Python: 3.11.13
ROOT: D:\IIT BBS\Job Resources\Business Optima\pdf-agent


### **Utilities: run one turn (streaming in notebook) + small log viewers**

In [ ]:
async def stream_once(agent: Title17Agent, session_id: str, user_text: str):
    print(f"[user] {user_text}\n")
    chunks = []
    async for ev in agent.achat_stream(session_id, user_text):
        if ev["type"] == "token":
            print(ev["text"], end="", flush=True)
            chunks.append(ev["text"])
        elif ev["type"] == "final":
            print("\n\n[FINAL]\n", ev["text"])
            if ev.get("citations"):
                print("\n[CITATIONS]")
                for c in ev["citations"]:
                    print(c)
        elif ev["type"] == "error":
            print("\n[ERROR]", ev["text"])
    return "".join(chunks)

def show_sqlite_summary():
    con = sqlite3.connect(CFG.sqlite_path)
    cur = con.cursor()
    for table in ["conversations", "messages", "events"]:
        try:
            n = cur.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
            print(f"{table}: {n} rows")
        except Exception as e:
            print(f"{table}: (not found) {e}")
    print("\n[last 10 events]")
    try:
        for r in cur.execute("SELECT ts,event,substr(payload,1,200) FROM events ORDER BY id DESC LIMIT 10"):
            print(r)
    except Exception as e:
        print("(no events) ->", e)
    con.close()

def show_session_json(session_id: str, last_n=8):
    fp = CFG.sessions_dir / f"{session_id}.json"
    if not fp.exists():
        print("[no session json]", fp)
        return
    data = json.loads(fp.read_text(encoding="utf-8"))
    msgs = data.get("messages", [])[-last_n:]
    print(f"[session file] {fp}  (last {len(msgs)} msgs)")
    for m in msgs:
        ts = time.strftime("%H:%M:%S", time.localtime(m["ts"]))
        print(f"- {ts} {m['role']}: {m['content'][:160]}")

### **Sequential chat stress test**

In [3]:
agent = Title17Agent()
sid = f"sess-{uuid.uuid4().hex[:8]}"
print("[session]", sid)

tests = [
    # easy
    "What are the four fair use factors in §107? End with [pp. 40–41].",

    # typo / noisy
    "wat does sectn 107 say abt fair use? end wth [pp. 40-41]",

    # context reference (ask something, then refer back)
    "Now compare that with §110 classroom exemptions. Short contrast + page refs.",
    "Given your last answer, how do §108 library rules interact with §107? End with page ranges.",

    # heavy, multi-part
    ("I need (1) the §107 factors, (2) a brief compare/contrast with §110 exemptions, and "
     "(3) a 4-bullet summary of §114’s performance-rights caveat. End with page ranges."),

    # out-of-scope
    "write a python crawler for reddit and summarize 10 threads",

    # random chit-chat (should keep Title 17 identity)
    "what's your favorite movie?",
]

# Jupyter supports top-level await
for q in tests:
    print("\n" + "="*88 + "\n")
    await stream_once(agent, sid, q)

print("\n" + "="*88)
print("SQLite + session JSON snapshots:\n")
show_sqlite_summary()
print()
show_session_json(sid, last_n=12)

D:\IIT BBS\Job Resources\Business Optima\pdf-agent\src\agent\memory.py:60: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=CFG.ollama_summarizer, temperature=0.2)
D:\IIT BBS\Job Resources\Business Optima\pdf-agent\src\agent\memory.py:61: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  return ConversationSummaryBufferMemory(


[session] sess-58d060e1


[user] What are the four fair use factors in §107? End with [pp. 40–41].

 Answer:
The four fair use factors are:

- The purpose and character of the use, including commercial vs. noncommercial use;
- The nature of the work being used;
- The amount and significance of the portion used relative to the whole; and
- The effect of the use on the potential market for or value of the work.

These factors are found in §107, which also provides guidelines for determining fair use.

[FINAL]
 Answer:
The four fair use factors are:

- The purpose and character of the use, including commercial vs. noncommercial use;
- The nature of the work being used;
- The amount and significance of the portion used relative to the whole; and
- The effect of the use on the potential market for or value of the work.

These factors are found in §107, which also provides guidelines for determining fair use.

[CITATIONS]
{'chunk_id': 'title17-h-193', 'node_id': 'SEC-00010', 'section': '', '

### **Gradio UI (streaming tokens, new session, export transcript)**

In [2]:
from src.ui.chat import start_app
demo = start_app()
demo.launch(server_name="127.0.0.1", server_port=7861, share=False)

D:\IIT BBS\Job Resources\Business Optima\pdf-agent\src\agent\memory.py:60: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=CFG.ollama_summarizer, temperature=0.2)
D:\IIT BBS\Job Resources\Business Optima\pdf-agent\src\agent\memory.py:61: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  return ConversationSummaryBufferMemory(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
